In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
df = pd.read_csv('E:/news.csv')

In [3]:
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6335 non-null   int64 
 1   title       6335 non-null   object
 2   text        6335 non-null   object
 3   label       6335 non-null   object
dtypes: int64(1), object(3)
memory usage: 198.1+ KB


In [6]:
data = df.drop(['Unnamed: 0', 'title'], axis=1)

In [7]:
data.head()

,text,label
0,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,It's primary day in New York and front-runners...,REAL


In [9]:
from sklearn.model_selection import train_test_split
corpus, test_corpus, y, yt = train_test_split(data.iloc[:,0], data.iloc[:,1], test_size=0.25, random_state=101)

In [10]:
from sklearn.feature_extraction import text
vectorizer = text.CountVectorizer(ngram_range=(1,2), stop_words='english').fit(corpus)

In [11]:
TfidF = text.TfidfTransformer()

In [13]:
X = TfidF.fit_transform(vectorizer.transform(corpus))
Xt = TfidF.transform(vectorizer.transform(test_corpus))

In [15]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

In [16]:
param_grid = {'C' : [0.01, 0.1, 1.0, 5.0, 10.0]}
clf = GridSearchCV(LinearSVC(loss='hinge', random_state=101), param_grid)

In [17]:
clf = clf.fit(X,y)

C:\Users\hp\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\hp\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [30]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
solution = clf.predict(Xt)
print("achieved accuracy: %0.3f" % accuracy_score(yt,solution))

achieved accuracy: 0.933


In [20]:
print(test_corpus[yt != solution])

3592    London is kidding itself if it thinks this wil...
589     First Read is a morning briefing from Meet the...
2204    A documentary about campus rape contains the d...
5713    BNI Store Nov 4 2016 “MODERATE” INDONESIA: Pro...
5924    Videos 30 Civilians Die In US Airstrike Called...
                              ...                        
137     Archives Michael On Television 10 Things That ...
3304    A public service announcement from your friend...
4066    The Next Big Shoe to Drop Posted on   The Next...
3870    Pick Your 2016 GOP Candidate -- Let Rush Know!...
3293    Why isn’t the Democratic Party landsliding the...
Name: text, Length: 106, dtype: object


In [23]:
print(test_corpus[yt == solution])

678     There is an path for Democrats to regain the p...
4325    Washington (CNN) Texas Sen. Ted Cruz is standi...
847     Sputnik October 27, 2016 \nNATO and Washington...
559     NGOs should condemn terrorists in Syria, not R...
6167    Republican groups are moving to the next phase...
                              ...                        
1867    Everyone getting hammered tonight for bad reas...
3365    Obama Admits to Rigged Elections back in 2008 ...
338     Netflix Ceo: TV’s Future includes Hallucinatio...
3753    It sounds corny to say it like that, but I don...
1564    A couple of weeks ago, I wrote a mostly faceti...
Name: text, Length: 1478, dtype: object


In [32]:
cm = confusion_matrix(yt,solution)
clfrep = classification_report(yt,solution)

print(cm)
print('/n')
print(clfrep)

[[717  51]
 [ 55 761]]
/n
              precision    recall  f1-score   support

        FAKE       0.93      0.93      0.93       768
        REAL       0.94      0.93      0.93       816

    accuracy                           0.93      1584
   macro avg       0.93      0.93      0.93      1584
weighted avg       0.93      0.93      0.93      1584

